In [21]:
import numpy as np
from scipy.io import loadmat,savemat
hsi_path = "D:\\HSI Project\\Updated_Work\\HSI_Classification\\Minerals_Dataset\\Minerals_mat_files\\0020_A.mat"

In [3]:
import torch
a = torch.tensor([[1,2],[3,4]])
a.element_size()*a.numel()),a.dtype

(32, torch.int64)

In [78]:
# %%time
# np.save(file="E:\\demo_patches_data\\pc2\\dummy_array.npy",arr=np.array(loadmat(hsi_path)['HDR']))
# for i in range(256):
#     np.array(loadmat(hsi_path)['HDR'])

CPU times: total: 17.5 s
Wall time: 40 s


In [54]:
import h5py
def patch_from_hsi(id,hsi_path,mmaps,window_size):
    hsi = mmaps[hsi_path]
    # hsi = np.transpose(loadmat(hsi_path)['HDR'],axes=(1,2,0))
    # hsi = np.transpose(h5py.File(hsi_path,'r')['HDR'][:],axes=(1,2,0))
    # hsi = self.hsi_dict[hsi_path]
    row,col,channels = hsi.shape
    x_hsi,y_hsi = id//col,id%col
    margin = int((window_size-1)/2)
    
    patch = np.zeros((window_size,window_size,channels))
    
    lower_bound = [max(0,x_hsi-margin),max(0,y_hsi-margin)]
    upper_bound = [min(row-1,x_hsi+margin),min(col-1,y_hsi+margin)]
    
    diff_lower = [abs(x_hsi-lower_bound[0]),abs(y_hsi-lower_bound[1])]
    diff_upper = [abs(upper_bound[0]-x_hsi),abs(upper_bound[1]-y_hsi)]
    
    patch_lower_bound = [margin-diff_lower[0],margin-diff_lower[1]]
    patch_upper_bound = [margin+diff_upper[0],margin+diff_upper[1]]
    
    patch[patch_lower_bound[0]:patch_upper_bound[0]+1,patch_lower_bound[1]:patch_upper_bound[1]+1,:] = hsi[lower_bound[0]:upper_bound[0]+1,lower_bound[1]:upper_bound[1]+1,:]
    del hsi
    return patch

In [46]:
from glob import glob
paths = glob("D:\\HSI Project\\Updated_Work\\HSI_Classification\\Minerals_Dataset\\Minerals_mat_files\\*.mat")
hsi_paths = []
for path in paths:
    name = path.split("\\")[-1]
    if "gt" not in name:
        hsi_paths.append(path)
hsi_dict = {}
for path in hsi_paths:
    hsi_dict[path] = np.transpose(loadmat(path)['HDR'][:],axes=(1,2,0)).shape
print(hsi_dict)

{'D:\\HSI Project\\Updated_Work\\HSI_Classification\\Minerals_Dataset\\Minerals_mat_files\\0000_A.mat': (320, 410, 256), 'D:\\HSI Project\\Updated_Work\\HSI_Classification\\Minerals_Dataset\\Minerals_mat_files\\0001_A.mat': (320, 410, 256), 'D:\\HSI Project\\Updated_Work\\HSI_Classification\\Minerals_Dataset\\Minerals_mat_files\\0002_A.mat': (320, 410, 256), 'D:\\HSI Project\\Updated_Work\\HSI_Classification\\Minerals_Dataset\\Minerals_mat_files\\0003_A.mat': (320, 410, 256), 'D:\\HSI Project\\Updated_Work\\HSI_Classification\\Minerals_Dataset\\Minerals_mat_files\\0003_B.mat': (320, 410, 256), 'D:\\HSI Project\\Updated_Work\\HSI_Classification\\Minerals_Dataset\\Minerals_mat_files\\0004_A.mat': (320, 410, 256), 'D:\\HSI Project\\Updated_Work\\HSI_Classification\\Minerals_Dataset\\Minerals_mat_files\\0005_A.mat': (320, 435, 256), 'D:\\HSI Project\\Updated_Work\\HSI_Classification\\Minerals_Dataset\\Minerals_mat_files\\0005_B.mat': (320, 435, 256), 'D:\\HSI Project\\Updated_Work\\HSI_Cla

In [52]:
mmaps = {}
for path,shape in hsi_dict.items():
    mmaps[path] = np.memmap(path, dtype='float32', mode='r', shape=shape)

In [53]:
len(mmaps.keys())

153

In [50]:
# mmaps[0]

In [57]:
%%time
id = np.arange(0,512)
a = torch.zeros((512,256,11,11))
# hsi = np.transpose(loadmat(hsi_path)['HDR'],axes=(1,2,0))
for i in id:
    patch = torch.tensor(np.transpose(patch_from_hsi(i,hsi_path,mmaps,11),axes=(2,0,1)))
    a[i] = patch

CPU times: total: 15.6 ms
Wall time: 65.3 ms


In [45]:
hsi.shape

(320, 410, 256)

In [37]:
a.shape

torch.Size([512, 256, 7, 7])

In [38]:
np.save("D:\\HSI Project\\Updated_Work\\HSI_Classification\\Vision_Transformer\\HSI_Transformer\\dummy2.npy",arr=a.numpy())

In [35]:
%%time
import torch
import torch.nn.functional as F
patch_path = "E:\\Patches_Data\\Patches\\0_A_0.npy"
id = np.arange(0,512)
b = torch.zeros((512,256,7,7))
for i in id:
    # patch = np.transpose(patch_from_hsi(i,hsi_path,11),axes=(2,0,1))
    p2 = np.load(patch_path)
    p2 = np.transpose(F.pad(torch.tensor(np.transpose(p2,axes=(2,0,1))),(1,1,1,1)).numpy(),axes=(1,2,0))
    p2 = torch.tensor(p2).permute(2,0,1)
    b[i] = p2

CPU times: total: 78.1 ms
Wall time: 105 ms


In [36]:
b.shape

torch.Size([512, 256, 7, 7])

In [39]:
np.save("D:\\HSI Project\\Updated_Work\\HSI_Classification\\Vision_Transformer\\HSI_Transformer\\dummy3.npy",arr=b.numpy())

In [32]:
b.dtype

torch.float32

In [14]:
arr = np.array(loadmat(hsi_path)['HDR'])
# arr.dtype # f4 means 4 bytes or 32 bits
arr.shape

(256, 320, 410)

In [18]:
arr[:,:11,:11].dtype

dtype('<f4')

In [17]:
np.save(file="D:\\HSI Project\\Updated_Work\\HSI_Classification\\Vision_Transformer\\HSI_Transformer\\dummy.npy",arr = arr[:,:11,:11])

In [7]:
import torch
print(torch.cuda.is_available())
print(torch.version.cuda)

True
11.8


In [4]:
print(hsi.shape)

(320, 410, 256)


In [13]:
def patch_from_hsi(id,hsi,window_size):
    row,col,channels = hsi.shape
    x_hsi,y_hsi = id//col,id%col
    margin = int((window_size-1)/2)
    
    patch = np.zeros((window_size,window_size,channels))
    
    lower_bound = [max(0,x_hsi-margin),max(0,y_hsi-margin)]
    upper_bound = [min(row-1,x_hsi+margin),min(col-1,y_hsi+margin)]
    
    diff_lower = [abs(x_hsi-lower_bound[0]),abs(y_hsi-lower_bound[1])]
    diff_upper = [abs(upper_bound[0]-x_hsi),abs(upper_bound[1]-y_hsi)]
    
    patch_lower_bound = [margin-diff_lower[0],margin-diff_lower[1]]
    patch_upper_bound = [margin+diff_upper[0],margin+diff_upper[1]]
    
    patch[patch_lower_bound[0]:patch_upper_bound[0]+1,patch_lower_bound[1]:patch_upper_bound[1]+1,:] = hsi[lower_bound[0]:upper_bound[0]+1,lower_bound[1]:upper_bound[1]+1,:]
    
    return patch

In [35]:
patch = patch_from_hsi(131199,hsi,5)
print(patch.shape)

(5, 5, 256)


In [32]:
patch[:,:,0]

array([[0.09467328, 0.10506839, 0.11166861, 0.        , 0.        ],
       [0.08470266, 0.0939758 , 0.09359382, 0.        , 0.        ],
       [0.08646677, 0.0933903 , 0.09653773, 0.        , 0.        ],
       [0.        , 0.        , 0.        , 0.        , 0.        ],
       [0.        , 0.        , 0.        , 0.        , 0.        ]])

In [33]:
131199//410,131199%410

(319, 409)

In [34]:
hsi[317:320,407:410,0]

array([[0.09467328, 0.10506839, 0.11166861],
       [0.08470266, 0.0939758 , 0.09359382],
       [0.08646677, 0.0933903 , 0.09653773]], dtype=float32)

In [6]:
import torch

# Example input batch of size (batch_size, 256, 5, 5)
# Let's simulate it with a random tensor with some NaN values
batch_size = 10
inputs = torch.randn(batch_size, 256, 5, 5)

# Introduce some NaN values for testing
inputs[2, 100, 1, 3] = float('nan')
inputs[5, 150, 4, 2] = float('nan')

# Check where the NaN values are
nan_mask = torch.isnan(inputs)

# Get the indices of NaN values
nan_indices = torch.nonzero(nan_mask, as_tuple=False)
print(nan_indices)
# Loop through the results and print them
for idx in nan_indices:
    batch_idx, channel_idx, height_idx, width_idx = idx
    print(f"NaN found in batch {batch_idx}, channel {channel_idx}, at position ({height_idx}, {width_idx})")


tensor([[  2, 100,   1,   3],
        [  5, 150,   4,   2]])
NaN found in batch 2, channel 100, at position (1, 3)
NaN found in batch 5, channel 150, at position (4, 2)


In [4]:
import pandas as pd
from tqdm import tqdm
from scipy.io import loadmat
import numpy as np
import torch
df = pd.read_csv("D:\\HSI Project\\Updated_Work\\HSI_Classification\\Minerals_Dataset\\Minerals_Mapped.csv")

In [66]:
df.head()

,ID,Experiment,Mineral,HSI_Path,MASK_Path
0,20,A,Actinolite,D:\HSI Project\Updated_Work\HSI_Classification...,D:\HSI Project\Updated_Work\HSI_Classification...
1,21,A,Actinolite,D:\HSI Project\Updated_Work\HSI_Classification...,D:\HSI Project\Updated_Work\HSI_Classification...
2,64,A,Actinolite,D:\HSI Project\Updated_Work\HSI_Classification...,D:\HSI Project\Updated_Work\HSI_Classification...
3,107,A,Albite,D:\HSI Project\Updated_Work\HSI_Classification...,D:\HSI Project\Updated_Work\HSI_Classification...
4,25,A,Almandine,D:\HSI Project\Updated_Work\HSI_Classification...,D:\HSI Project\Updated_Work\HSI_Classification...


In [67]:
df.iloc[0]['MASK_Path']

'D:\\HSI Project\\Updated_Work\\HSI_Classification\\Minerals_Dataset\\Minerals_mat_files\\0020_A_gt.mat'

In [5]:
nan_img = []
for _,row in tqdm(df.iterrows(),total=len(df)):
    hsi_path = row['HSI_Path']
    hsi = torch.tensor(np.transpose(loadmat(hsi_path)['HDR'][:],axes=(1,2,0)))
    # Check where the NaN values are
    nan_mask = torch.isnan(hsi)
    # Get the indices of NaN values
    nan_indices = torch.nonzero(nan_mask, as_tuple=False)
    if len(nan_indices)>0:
        # print(row['ID']," ",row['Experiment'])
        # print(nan_indices,"\n")
        nan_img.append([row['ID'],row['Experiment'],nan_indices])

100%|██████████| 152/152 [00:10<00:00, 15.09it/s]


In [6]:
len(nan_img)

0

In [73]:
for i,x in enumerate(nan_img):
    print(i," ",x[:2])

In [40]:
for i in nan_img[14][2]:
    print(i)

tensor([0, 0, 0])
tensor([0, 0, 1])
tensor([0, 0, 2])
tensor([0, 0, 3])
tensor([0, 0, 4])
tensor([0, 0, 5])
tensor([0, 0, 6])
tensor([0, 0, 7])
tensor([0, 0, 8])
tensor([0, 0, 9])
tensor([ 0,  0, 10])
tensor([ 0,  0, 11])
tensor([ 0,  0, 12])
tensor([ 0,  0, 13])
tensor([ 0,  0, 14])
tensor([ 0,  0, 15])
tensor([ 0,  0, 16])
tensor([ 0,  0, 17])
tensor([ 0,  0, 18])
tensor([ 0,  0, 19])
tensor([ 0,  0, 20])
tensor([ 0,  0, 21])
tensor([ 0,  0, 22])
tensor([ 0,  0, 23])
tensor([ 0,  0, 24])
tensor([ 0,  0, 25])
tensor([ 0,  0, 26])
tensor([ 0,  0, 27])
tensor([ 0,  0, 28])
tensor([ 0,  0, 29])
tensor([ 0,  0, 30])
tensor([ 0,  0, 31])
tensor([ 0,  0, 32])
tensor([ 0,  0, 33])
tensor([ 0,  0, 34])
tensor([ 0,  0, 35])
tensor([ 0,  0, 36])
tensor([ 0,  0, 37])
tensor([ 0,  0, 38])
tensor([ 0,  0, 39])
tensor([ 0,  0, 40])
tensor([ 0,  0, 41])
tensor([ 0,  0, 42])
tensor([ 0,  0, 43])
tensor([ 0,  0, 44])
tensor([ 0,  0, 45])
tensor([ 0,  0, 46])
tensor([ 0,  0, 47])
tensor([ 0,  0, 48])
t

In [46]:
def spatial_avg(data,coordinates,kernel_size=3):
    (x,y,c) = coordinates
    # data = patch[:,:,c].copy()
    while (np.isnan(data[x,y]) and kernel_size<=5):
        max_x = data.shape[0]-1
        max_y = data.shape[1]-1
        start_x = max(0,x-kernel_size//2)
        start_y = max(0,y-kernel_size//2)
        end_x = min(max_x,x+kernel_size//2)+1
        end_y = min(max_y,y+kernel_size//2)+1
        neighbours = []
        for i in range(start_x,end_x):
            for j in range(start_y,end_y):
                neighbours.append(data[i,j])
        # Check if all elements are nan
        all_nan = all(np.isnan(x) for x in neighbours)
        if all_nan:
            kernel_size+=2
            continue
        data[x,y] = np.nanmean(neighbours)
    return data

def fill_nan_hdr(hsi_img):
    # hsi_img: shape (h,w,c)
    hsi = hsi_img.copy()
    ind = np.transpose(np.where(np.isnan(hsi)))
    ind = np.array(sorted(ind,key=lambda x: (x[2],x[0],x[1])))
    for (x,y,c) in ind:
        if (c==0):
            if not np.isnan(hsi[x,y,c+1]):
                hsi[x,y,c] = hsi[x,y,c+1]
            else:
                hsi[:,:,c] = spatial_avg(hsi[:,:,c],(x,y,c))
        elif (c==255):
            if not np.isnan(hsi[x,y,c-1]):
                hsi[x,y,c] = hsi[x,y,c-1]
            else:
                hsi[:,:,c] = spatial_avg(hsi[:,:,c],(x,y,c))
        else:
            if (not np.isnan(hsi[x,y,c+1])) and (not np.isnan(hsi[x,y,c-1])):
                hsi[x,y,c] = np.mean([hsi[x,y,c+1],hsi[x,y,c-1]])
            elif not np.isnan(hsi[x,y,c-1]):
                hsi[x,y,c] = hsi[x,y,c-1]
            elif not np.isnan(hsi[x,y,c+1]):
                hsi[x,y,c] = hsi[x,y,c+1]
            else:
                hsi[:,:,c] = spatial_avg(hsi[:,:,c],(x,y,c))
    return hsi

In [41]:
from glob import glob
import h5py 

In [42]:
hdr_files = list(glob("D:\\HSI Project\\Updated_Work\\HSI_Classification\\Minerals_Dataset\\HDR_Data\\Extracted_Folder\\*\\*.hdr.h5"))
hdr_files[0]

'D:\\HSI Project\\Updated_Work\\HSI_Classification\\Minerals_Dataset\\HDR_Data\\Extracted_Folder\\0000\\A.hdr.h5'

In [62]:
hdr_path = "D:\\HSI Project\\Updated_Work\\HSI_Classification\\Minerals_Dataset\\HDR_Data\\Extracted_Folder\\0089\\A.hdr.h5"

In [63]:
hsi = h5py.File(hdr_path,'r')['hdr'][:] # (256,320,410)
        
## Handling Nan Values here 
hsi = np.transpose(hsi,axes=(1,2,0))
hsi = fill_nan_hdr(hsi)
hsi = np.transpose(hsi,axes=(2,0,1))  # (256,320,410)

In [64]:
nan_mask = torch.isnan(torch.tensor(hsi))
# Get the indices of NaN values
nan_indices = torch.nonzero(nan_mask, as_tuple=False)

In [65]:
nan_indices

tensor([[  0,   0,   0],
        [  1,   0,   0],
        [  2,   0,   0],
        [  3,   0,   0],
        [  4,   0,   0],
        [  5,   0,   0],
        [  6,   0,   0],
        [  7,   0,   0],
        [  8,   0,   0],
        [  9,   0,   0],
        [ 10,   0,   0],
        [ 11,   0,   0],
        [ 12,   0,   0],
        [ 13,   0,   0],
        [ 14,   0,   0],
        [ 15,   0,   0],
        [ 16,   0,   0],
        [ 17,   0,   0],
        [ 18,   0,   0],
        [ 19,   0,   0],
        [ 20,   0,   0],
        [ 21,   0,   0],
        [ 22,   0,   0],
        [ 23,   0,   0],
        [ 24,   0,   0],
        [ 25,   0,   0],
        [ 26,   0,   0],
        [ 27,   0,   0],
        [ 28,   0,   0],
        [ 29,   0,   0],
        [ 30,   0,   0],
        [ 31,   0,   0],
        [ 32,   0,   0],
        [ 33,   0,   0],
        [ 34,   0,   0],
        [ 35,   0,   0],
        [ 36,   0,   0],
        [ 37,   0,   0],
        [ 38,   0,   0],
        [ 39,   0,   0],
